In [1]:
import tensorflow as tf

In [2]:
example = tf.train.Example()

In [3]:
example

In [112]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(values):
  """Returns a bytes_list from a string / byte."""
  vv = values
  if isinstance(values[0], type(tf.constant(0))):
    vv = [val.numpy() for val in values]
#     value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=vv))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [109]:
!ls

Beauty.txt     Untitled.ipynb ml1m.txt
Steam.txt      Video.txt      tfrecord.ipynb


In [110]:
file1 = open('ml1m.txt', 'r')
lines = file1.readlines()
# lines

In [113]:
ms = [b'user_id1', b'user_id2']
# ms = ms.encode('UTF-8')
a = _bytes_feature(ms)

# print(isinstance(ms, type(tf.constant(0))))
a

bytes_list {
  value: "user_id1"
  value: "user_id2"
}

In [29]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'user': _bytes_feature(feature0),
      'video': _bytes_feature(feature1),
      'values': _float_feature(feature2),
      'weights': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [31]:
serialized_example = serialize_example(b'user_id1', b'item_id0', 1.0, 1.0)
serialized_example

b'\nV\n\x13\n\x07weights\x12\x08\x12\x06\n\x04\x00\x00\x80?\n\x15\n\x05video\x12\x0c\n\n\n\x08item_id0\n\x14\n\x04user\x12\x0c\n\n\n\x08user_id1\n\x12\n\x06values\x12\x08\x12\x06\n\x04\x00\x00\x80?'

In [32]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

features {
  feature {
    key: "user"
    value {
      bytes_list {
        value: "user_id1"
      }
    }
  }
  feature {
    key: "values"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "video"
    value {
      bytes_list {
        value: "item_id0"
      }
    }
  }
  feature {
    key: "weights"
    value {
      float_list {
        value: 1.0
      }
    }
  }
}

In [26]:
for line in lines:
    a = line.split(' ')
    user_id = 'user_id' + a[0]
    item_id = 'item_id' + a[1]
    
    bf = _bytes_feature(item_id.encode('UTF-8'))
    example.features.feature['user']

user_id1 item_id32

user_id1 item_id23

user_id1 item_id28

user_id1 item_id38

user_id1 item_id25



In [88]:
myd = {
    'user' : ['user_id1'],
    'video': ['video_id3', 'video_id6'],
    'values' : [1.0, 1.0],
    'weights' : [1.0, 1.0],
}

In [89]:
def create_serialized_example(name_to_values):

    example = tf.train.Example()
    for name, values in name_to_values.items():
        feature = example.features.feature[name]
        if isinstance(values[0], str):
            for i in range(len(values)):
                values[i] = values[i].encode('UTF-8')
            add = feature.bytes_list.value.extend
        elif isinstance(values[0], float):
            add = feature.float32_list.value.extend
        elif isinstance(values[0], int):
            add = feature.int64_list.value.extend
        else:
            raise AssertionError('Unsupported type: %s' % type(values[0]))

    add(values)
    return example.SerializeToString()

In [82]:
def cse(md):
    example = tf.train.Example()
    for name, values in md.items():
        feature = example.features.feature[name]
        if isinstance(values[0], str):
            for i in range(len(values)):
                values[i] = values[i].encode('UTF-8')
            add = feature.bytes_list.value.extend
        elif isinstance(values[0], float):
            add = feature.float32_list.value.extend
        elif isinstance(values[0], int):
            add = feature.int64_list.value.extend
        else:
            raise AssertionError('unsupported type : %s' % type(values[0]))
        
        add(values)
    return example.SerializeToString()

In [90]:
create_serialized_example(myd)

AttributeError: float32_list

In [53]:
for name, values in myd.items():
    print(name, values[0])
    

user b'user_id1'
video b'video_id3'
values 1.0
weights 1.0
